#### Defined by User

In [ ]:
import os

# IP of the DKube setup without https://
# and without port number eg: 1.2.3.4
DKUBE_IP = "192.168.200.146"

## specify your Dkube username
DKUBEUSERNAME = os.getenv("DKUBE_USER_LOGIN_NAME","")

## Define the model monitor name here that you will be creating 
MONITOR_NAME = "image-mm-kf"

# Required in train.ipynb to retrain the model
DKUBE_TRAINING_CODE_NAME = "chest-xray"

LIVE_DATASET =  MONITOR_NAME+'-s3'

## Dkube information
TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","")
DKUBE_URL = os.getenv("DKUBE_URL","")


MINIO_ENDPOINT = f"http://{DKUBE_IP}:32221"

TRAINING_DATASET = 'chest-xray-data'
## Inference URL
INFERENCE_URL = None

# Model name to be created or used for example, it will create the model
# if not existing otherwise it will ignore creation.
MODEL_NAME = MONITOR_NAME

# the frequency with which monitoring will run, value will be considered in minutes
RUN_FREQUENCY = 5

if not(TOKEN or DKUBEUSERNAME or DKUBE_URL or DKUBE_IP):
    print("Please fill the Dkube details first (TOKEN, DKUBE_URL, DKUBEUSERNAME, DKUBE_IP)")
    raise TypeError

#### Dkube Resources

In [ ]:
import time,json
from dkube.sdk import *

In [ ]:
api = DkubeApi(URL=DKUBE_URL,token=TOKEN)
if DKUBEUSERNAME == api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, first")

In [ ]:
data = api.get_modelmonitor_id(MONITOR_NAME)
if data.data:
    MONITOR_ID = data.data.get(MONITOR_NAME)
    if MONITOR_ID:
        raise ValueError(f"{MONITOR_NAME} already existing please use a different name")

In [ ]:
response =  api.get_cloudevents_logstore_creds()
MINIO_KEY = response["access_key_id"]
MINIO_SECRET_KEY = response["access_key"]
MINIO_BUCKET = response["bucket"]

In [ ]:
INPUT_TRAIN_TYPE = "training"
image_exp_config = {"MONITOR_NAME":MONITOR_NAME,
                    "INPUT_TRAIN_TYPE":INPUT_TRAIN_TYPE, "DKUBEUSERNAME":DKUBEUSERNAME,
                    "TOKEN":TOKEN, "DKUBE_URL":DKUBE_URL, "MINIO_KEY":MINIO_KEY,
                    "MINIO_SECRET_KEY":MINIO_SECRET_KEY, "MINIO_ENDPOINT":MINIO_ENDPOINT,
                    "MODEL_NAME":MODEL_NAME,
                    "TRAINING_DATASET":TRAINING_DATASET, "RUN_FREQUENCY":RUN_FREQUENCY,
                    "DKUBE_TRAINING_CODE_NAME":DKUBE_TRAINING_CODE_NAME, "MINIO_BUCKET": MINIO_BUCKET,
                    "LIVE_DATASET":LIVE_DATASET}
%store image_exp_config

#### Dataset

In [ ]:
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name=TRAINING_DATASET)
    dataset.update_dataset_source(source="git")
    dataset.update_git_details(url="https://github.com/oneconvergence/dkube-examples/tree/monitoring/image_cloudevents/data/chest-xray-mini")
    api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

### Model Monitor Datasets

##### Predict and labelled Dataset

In [ ]:
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name=LIVE_DATASET,remote=True)
    dataset.update_dataset_source('s3')
    dataset.update_s3_details(
        endpoint = MINIO_ENDPOINT,
        bucket=MINIO_BUCKET,
        prefix='',
        key=MINIO_KEY,
        secret=MINIO_SECRET_KEY)
    api.create_dataset(dataset)

except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e


#### Model

In [ ]:
try:
    model = DkubeModel(DKUBEUSERNAME, name=MODEL_NAME)
    model.update_model_source(source='dvs')
    api.create_model(model)
except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    api.delete_dataset(DKUBEUSERNAME,TRAINING_DATASET,force=True)
    api.delete_dataset(DKUBEUSERNAME,S3_DATASET,force=True)
    api.delete_model(DKUBEUSERNAME,MODEL_NAME,force=True)
    %store -d image_exp_config